In [2]:
!pip3 install transformers
!pip3 install SentencePiece
import numpy
from transformers import T5Tokenizer, T5ForConditionalGeneration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00


In [27]:
model_name = "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/v2_OpenEnded_mscoco_train2014_questions.json', 'r') as file:
    annotations_data = json.load(file)

In [ ]:
annotations_data

In [28]:
def question_to_template(question, demonstrations):
    # A refined prompt, making the instruction more explicit
    input_text = f"{demonstrations} Based on the patterns, convert: '{question}' into a statement with a [mask]."

    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(input_ids)

    # Extract the generated template
    template = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return template

# Using the same demonstrations and question

demos = 'What is your name? Your name is [mask]. Where is the Eiffel Tower? The Eiffel Tower is in [mask]. '

question = "What color is the sky?"
template = question_to_template(question, demos)
print(template)

'What color is the sky?' into a statement with a [mas


In [11]:
# question = "What color is the apple?"
# examples = "What color is the car? The car color is [mask]. How many apples are there? THe number of apples are[mask]."

# template = convert_to_template(question, examples)
# print(template)

<pad> <extra_id_0> </s>
</s>


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [29]:
def get_filtered_answers(question_template, answer_vocabulary, k=5):
    # Get the likelihood of each answer fitting into the template
    likelihoods = {}
    for answer in answer_vocabulary:
        prompt = question_template.replace("[mask]", answer)
        input_tensor = tokenizer(prompt, return_tensors="pt").input_ids
        output = model.generate(input_tensor)
        decoded_output = tokenizer.decode(output[0])

        # A simple method: the likelihood is based on how close the model's output is to the input
        likelihoods[answer] = sum(1 for a, b in zip(decoded_output, prompt) if a == b)

    # Get top k answers based on likelihood
    top_k_answers = sorted(likelihoods, key=likelihoods.get, reverse=True)[:k]

    return top_k_answers

In [30]:
question_template = "The species of the flower is [mask]"
answer_template = ["rose","tulip","daisy","vase","table"]
top_answers = get_filtered_answers(question_template, answer_template,k=4)

In [31]:
top_answers

['daisy', 'rose', 'vase', 'tulip']

[mask].</s>
